In [ ]:
from common.database import get_db
from common.models import *

In [ ]:
from sqlalchemy import func, text, select

In [ ]:
import datetime as dt
import statistics

In [ ]:
yesterday = dt.datetime.now() - dt.timedelta(days=1)
start_dt = yesterday.replace(hour=0, minute=0, second=0)
end_dt = yesterday.replace(hour=23, minute=59, second=59)

In [ ]:
session = next(get_db())
created_leads: int = session.query(func.count(Lead.id)).filter(start_dt <= Lead.created_at, Lead.created_at <= end_dt).scalar()
print(created_leads)

In [ ]:
closed_lead_ids = (
    session
    .query(LeadStatusChange.lead_id)
    .filter(
        func.date(LeadStatusChange.created_at) == yesterday.date(), 
        LeadStatusChange.new_status_id == 142
    )
    .all()
)
closed_lead_ids = [row[0] for row in closed_lead_ids]
print(closed_lead_ids)

In [ ]:
closed_leads = (
    session
    .query(Lead)
    .filter(
        Lead.id.in_(closed_lead_ids)
    )
    .all()
)
print(len(closed_leads))
# for lead in closed_leads:
#     if lead.status_id != 142:
#         print(lead)

In [ ]:
sells = [lead.price for lead in closed_leads]

In [ ]:
total = sum(sells)
print(total)

In [ ]:
average = statistics.mean(sells)
print(average)

In [ ]:
conversion = created_leads / len(closed_leads) if len(closed_leads) else 0
print(conversion)